In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IMPORTING THE LIBRARIES





In [3]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, MaxPool1D, Dropout, Dense, GlobalMaxPooling1D, Embedding, Activation
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [5]:
import os
os.chdir("/content/drive/")
!ls

df = pd.read_csv('/content/drive/MyDrive/data/data.csv')

df.head()

actual_data = pd.DataFrame(columns = ['comment_text', 'toxic'])

for i in range(len(df)):
    text = df['tweet'].iloc[i]
    if df['class'].iloc[i] == 2:
        hate = 1
    else:
        hate = 0
        
    actual_data = actual_data.append({'comment_text' : text, 'toxic' : hate}, ignore_index = True) 


MyDrive


TRAINING AND TESTING THE DATA

In [6]:
train, test = train_test_split(actual_data, test_size=0.2, random_state=1)

In [7]:
train.head()

,comment_text,toxic
11746,Im prolly wit yo bitch big butt cant fit up in...,0
12823,Mei Haruka is tied up and takes three cocks in...,0
20436,RT @zachpiecowiak: @Eddie_Sativa87 you're a fu...,0
2182,.@CoryBooker is running around town delivering...,1
9288,Fuck these hoes im gone &#9996;&#65039;,0


In [8]:
test.head()

,comment_text,toxic
13932,Pumpkin spice Marlboro's for da hoes,0
1636,&#8220;@_CiaraaaS: What things do you love? &#...,0
23100,Yankees should have NEVER gave away Melky. Guy...,1
3498,@Im_Thirst I love Louis CK! Quit bein a faggot...,0
12999,My boyfriend is such a smart ass bitch watch y...,0


In [9]:
train_data = train
test_data = test

PREPROCESSING THE DATA

In [10]:
def preprocess_text(sen):
# lower the character
    sentence = sen.lower()
    
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    stops = stopwords.words('english')
    
    for word in sentence.split():
        if word in stops:
            sentence = sentence.replace(word, '')
    return sentence

TESTING AND TRAINING THE PREPROCESSED DATA


In [11]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
train_data['comment_text'] = train_data['comment_text'].apply(lambda x : preprocess_text(x))
test_data['comment_text'] = test_data['comment_text'].apply(lambda x : preprocess_text(x))

In [13]:
train_data.head()

,comment_text,toxic
11746,Im prolly wit yo bitch big butt cant fit,0
12823,Mei Haruka tied takes three cocks pussy ...,0
20436,RT zachpiecowiak Eddie Sativa fucking queer,0
2182,CoryBooker running around wn delivering food...,1
9288,Fuck hoes im gone,0


TOKENIZING THE DATA

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
token = Tokenizer()
token.fit_on_texts(train_data['comment_text'])
seq = token.texts_to_sequences(train_data['comment_text'])

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
pad_seq = pad_sequences(seq,maxlen=200)

In [17]:
pad_seq

array([[    0,     0,     0, ...,   946,   360,  1511],
       [    0,     0,     0, ...,     4,     3, 11393],
       [    0,     0,     0, ...,  7255,    33,   378],
       ...,
       [    0,     0,     0, ...,    11,   234,  1635],
       [    0,     0,     0, ...,   492,   334, 32990],
       [    0,     0,     0, ...,     6,    12,    16]], dtype=int32)

In [18]:
y = train_data['toxic'].values

In [19]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

In [20]:
vocab_size = len(token.word_index)+1

In [21]:
print(vocab_size)

32994


SPLITTING THE DATA INTO TRAINING AND TESTING DATA

In [22]:
X_train, X_test, y_train, y_test = train_test_split(pad_seq, y, test_size=0.2, random_state=1)

BUILDING THE MODEL

In [23]:
from tqdm import tqdm 

In [24]:
from tqdm import tqdm
embedding_vector = {}
f = open('/content/drive/MyDrive/data/glove.6B.200d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400000it [00:23, 17060.58it/s]


In [25]:
embedding_matrix = np.zeros((vocab_size,200))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 32993/32993 [00:00<00:00, 268632.76it/s]


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout,Embedding,Bidirectional

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, 200, weights = [embedding_matrix], input_length = 200, trainable = False))

model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Bidirectional(LSTM(32)))

model.add(Dropout(0.5))

model.add(Dense(256,activation = 'tanh'))
model.add(Dropout(0.5))

model.add(Dense(7,activation = 'softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          6598800   
                                                                 
 bidirectional (Bidirectiona  (None, 200, 64)          59648     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0

TRAINING THE MODEL

In [29]:
import tensorflow as tf
pad_seq = tf.convert_to_tensor(pad_seq, dtype=tf.int64) 

y = tf.convert_to_tensor(y, dtype=tf.int64) 

In [30]:
history = model.fit(pad_seq, y ,epochs = 5, batch_size=256 ,validation_split=0.2, verbose = 1)

Epoch 1/5
62/62 [==============================] - 89s 1s/step - loss: 0.5682 - accuracy: 0.8279 - val_loss: 0.2324 - val_accuracy: 0.9012
Epoch 2/5
62/62 [==============================] - 80s 1s/step - loss: 0.2105 - accuracy: 0.9120 - val_loss: 0.1798 - val_accuracy: 0.9196
Epoch 3/5
62/62 [==============================] - 78s 1s/step - loss: 0.1752 - accuracy: 0.9254 - val_loss: 0.1645 - val_accuracy: 0.9244
Epoch 4/5
62/62 [==============================] - 80s 1s/step - loss: 0.1529 - accuracy: 0.9402 - val_loss: 0.1602 - val_accuracy: 0.9307
Epoch 5/5
62/62 [==============================] - 80s 1s/step - loss: 0.1376 - accuracy: 0.9448 - val_loss: 0.1825 - val_accuracy: 0.9254


PREDICTING THE OUTPUT

In [31]:
sentence = 'i had a bad day'

sent_seq = token.texts_to_sequences([sentence])

sent_pad_seq = pad_sequences(sent_seq ,maxlen=200)
pad_seq_tf = tf.convert_to_tensor(sent_pad_seq, dtype=tf.int64) 

predictions = model.predict(pad_seq_tf)
pred = np.argmax(predictions)

In [32]:
pred

0

In [33]:
sentence = 'nice to meet u'

sent_seq = token.texts_to_sequences([sentence])

sent_pad_seq = pad_sequences(sent_seq ,maxlen=200)
pad_seq_tf = tf.convert_to_tensor(sent_pad_seq, dtype=tf.int64) 

predictions = model.predict(pad_seq_tf)
pred = np.argmax(predictions)

In [34]:
pred

0